In [1]:
with open('my_file', 'w') as fp:
    data = fp.write("Hello world")

In [2]:
print(fp.__enter__)
print(fp.__exit__)

<built-in method __enter__ of _io.TextIOWrapper object at 0x0000000004BDC8B8>
<built-in method __exit__ of _io.TextIOWrapper object at 0x0000000004BDC8B8>


In [5]:
class ContextManager(object):
    
    def __enter__(self):
        print("Entering")
    
    def __exit__(self, exc_type, exc_value, traceback):
         print("Entering")

In [7]:
with ContextManager():
    print(" Inside the with statement")

Entering
 Inside the with statement
Entering


In [9]:
with ContextManager():
    print(1/0)

Entering
Entering


ZeroDivisionError: division by zero

####  __enter__返回值

In [10]:
class ContextManager(object):
    
    def __enter__(self):
        print("Entering")
        return "my value"
    def __exit__(self, exc_type, exc_value, traceback):
        print("Exiting")

In [13]:
with ContextManager() as value:
    print(value)

Entering
my value
Exiting


#####  一个通常的做法是将 __enter__ 的返回值设为这个上下文管理器对象本身

In [15]:
fp = open('my_file', 'r')
print(fp.__enter__())
fp.close()

<_io.TextIOWrapper name='my_file' mode='r' encoding='cp936'>


In [16]:
import os
os.remove('my_file')

In [17]:
class ContextManager(object):
    
    def __enter__(self):
        print("Entering")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Exiting")

In [18]:
with ContextManager() as value:
    print(value)

Entering
Exiting


In [2]:
class ContextManager(object):
    
    def __enter__(self):
        print("Entering")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Exiting")
        if exc_type is not None:
            print("  Exception:", exc_value)
            return True#增加return True后就不会报错

with ContextManager():
    print(1/0)

Entering
Exiting
  Exception: division by zero


##### 使用__init__方法给一个类传递参数

In [3]:
class Transaction(object):
    
    def __init__(self, connection):
        self.connection = connection
    
    def __enter__(self):
        return self.connection.cursor()
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_value is None:
            # transaction was OK, so commit
            self.connection.commit()
        else:
            # transaction had a problem, so rollback
            self.connection.rollback()

In [4]:
import sqlite3 as db
connection = db.connect(":memory:")

with Transaction(connection) as cursor:
    cursor.execute("""CREATE TABLE IF NOT EXISTS addresses (
        address_id INTEGER PRIMARY KEY,
        street_address TEXT,
        city TEXT,
        state TEXT,
        country TEXT,
        postal_code TEXT
    )""")

In [5]:
with Transaction(connection) as cursor:
    cursor.executemany("""INSERT OR REPLACE INTO addresses VALUES (?, ?, ?, ?, ?, ?)""", [
        (0, '515 Congress Ave', 'Austin', 'Texas', 'USA', '78701'),
        (1, '245 Park Avenue', 'New York', 'New York', 'USA', '10167'),
        (2, '21 J.J. Thompson Ave.', 'Cambridge', None, 'UK', 'CB3 0FA'),
        (3, 'Supreme Business Park', 'Hiranandani Gardens, Powai, Mumbai', 'Maharashtra', 'India', '400076'),
    ])

In [6]:
with Transaction(connection) as cursor:
    cursor.execute("""INSERT OR REPLACE INTO addresses VALUES (?, ?, ?, ?, ?, ?)""",
        (4, '2100 Pennsylvania Ave', 'Washington', 'DC', 'USA', '78701'),
    )
    raise Exception("out of addresses")

Exception: out of addresses

#####  产生问题的最新插入将不会被保存，而是返回上一次commit成功的状态

In [7]:
cursor.execute("SELECT * FROM addresses")
for row in cursor:
    print(row)

(0, '515 Congress Ave', 'Austin', 'Texas', 'USA', '78701')
(1, '245 Park Avenue', 'New York', 'New York', 'USA', '10167')
(2, '21 J.J. Thompson Ave.', 'Cambridge', None, 'UK', 'CB3 0FA')
(3, 'Supreme Business Park', 'Hiranandani Gardens, Powai, Mumbai', 'Maharashtra', 'India', '400076')


#### python3特性  urllib.urlopen()----urllib.request.urlopen()

In [10]:
from contextlib import closing
import urllib.request

with closing(urllib.request.urlopen('http://www.baidu.com')) as url:
    html = url.read()

print(html[:100])

b'<!DOCTYPE html>\n<!--STATUS OK-->\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r'


#### 使用修饰符 @contextlib
contextmanager 这个装饰器接受一个 generator，用 yield 语句把 with ... as var 把变量输出出去，然后，with 语句就可以正常的工作了


代码的执行顺序是：

 with 语句 首先执行 yield 之前的语句
 yield 调用会执行 with 语句内部的所有语句
 最后执行yield之后的语句

In [11]:
from contextlib import contextmanager

@contextmanager
def my_contextmanager():
    print("Enter")
    yield
    print("Exit")

with my_contextmanager():
    print("  Inside the with statement")

Enter
  Inside the with statement
Exit


In [12]:
@contextmanager
def my_contextmanager():
    print("Enter")
    yield "my value"
    print("Exit")

with my_contextmanager() as value:
    print(value)

Enter
my value
Exit


In [13]:
@contextmanager
def my_contextmanager():
    print("Enter")
    try:
        yield
    except Exception as exc:
        print("   Error:", exc)
    finally:
        print("Exit")
        
with my_contextmanager():
    print(1/0)

Enter
   Error: division by zero
Exit


In [14]:
@contextmanager
def transaction(connection):
    cursor = connection.cursor()
    try:
        yield cursor
    except:
        connection.rollback()
        raise
    else:
        connection.commit()

In [16]:
class PythonContextManagerDemo:
    
    def __enter__(self):
        print("Entering")
    def __exit__(self, *unused):
        print("Exiting")
with PythonContextManagerDemo():
    print("In the block")

Entering
In the block
Exiting
